In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [4]:
df = pd.read_csv('../Dataset/cleaned_dataset.csv')

In [5]:
df.head()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,...,R_yn,spark,aws,excel,tableau,power bi,ml,dl,job_sim,seniority
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,...,0,0,0,1,1,1,1,0,data scientist,na
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,...,0,0,0,0,0,0,1,0,data scientist,na
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,...,0,1,0,1,0,0,1,0,data scientist,na
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,...,0,0,0,0,0,0,1,1,data scientist,na
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,...,0,0,0,1,0,0,1,0,data scientist,na


In [6]:
df.columns

Index(['Unnamed: 0', 'Job Title', 'Salary Estimate', 'Job Description',
       'Rating', 'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'Hourly', 'Employer Provided Salary', 'min_salary', 'max_salary',
       'avg_salary', 'Company_txt', 'job_state', 'same_state', 'age',
       'python_yn', 'R_yn', 'spark', 'aws', 'excel', 'tableau', 'power bi',
       'ml', 'dl', 'job_sim', 'seniority'],
      dtype='object')

In [7]:
df_model = df[['avg_salary','Sector','job_state','python_yn','job_sim','R_yn','tableau','power bi','ml','dl']]


In [8]:
df_dum = pd.get_dummies(df_model) 

In [9]:
from sklearn.model_selection import train_test_split
X = df_dum.drop('avg_salary', axis = 1)
y = df_dum.avg_salary.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)


In [10]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor()

In [11]:
from sklearn.model_selection import cross_val_score

np.mean(cross_val_score(regressor, X_train, y_train, scoring = 'neg_mean_absolute_error', cv= 5))


-16.79919269560341

In [12]:
from sklearn.model_selection import GridSearchCV
parameters = {
    "n_estimators": range(10, 400, 10),
    "criterion": ['mse','mae'],
    "max_features": ['auto','sqrt','log2']
    }

gs = GridSearchCV(regressor, param_grid = parameters, scoring = 'neg_mean_absolute_error', cv = 5)
gs.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [13]:
gs.best_score_
y_pred = gs.best_estimator_.predict(X_test)

In [14]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)

12.975067351682501

In [15]:
import pickle
filename = '../Model/model.sav'
pickle.dump(gs.best_estimator_, open(filename, 'wb'))

# saving the columns
model_columns = list(X.columns)
with open('../Model/model_columns.pkl','wb') as file:
    pickle.dump(model_columns, file)
